In [1]:
from ydata_profiling import ProfileReport
import os
from sklearn.preprocessing import LabelEncoder
import wandb
import pandas as pd 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import math  
import numpy as np 

In [2]:
run = wandb.init(project="predict_house_price", job_type="download_pca_data")

artifact = run.use_artifact('pca_X_y_train_test:latest', type='processed_data')

artifact_dir = artifact.download()

X_train = pd.read_csv(os.path.join(artifact_dir, "X_train.csv"), index_col=0)
X_test = pd.read_csv(os.path.join(artifact_dir, "X_test.csv"), index_col=0)
y_train = pd.read_csv(os.path.join(artifact_dir, "y_train.csv"), index_col=0)
y_test = pd.read_csv(os.path.join(artifact_dir, "y_test.csv"), index_col=0)

run.finish()

wandb: Currently logged in as: kngocun (kngocun-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb:   4 of 4 files downloaded.  
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [3]:
model = xgb.XGBRegressor(
    n_estimators=500,    # số lượng cây
    learning_rate=0.05,   # tốc độ học
    max_depth=10,         # độ sâu tối đa mỗi cây
    random_state=42
)

model.fit(X_train, y_train)  # Train model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, ...)

In [4]:
y_pred = model.predict(X_test) 

In [5]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error: {rmse:.4f}")

Root Mean Squared Error: 40742.7174


In [6]:
run = wandb.init(project="predict_house_price", job_type="model_evaluation")

wandb.log({
    "RMSE": rmse,
    "y_pred": y_pred.tolist()  # Chuyển đổi y_pred sang list nếu cần (WandB không hỗ trợ numpy array trực tiếp)
})

run.finish()


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


RMSE,▁
RMSE,40742.71744
